# Assignment 6: Reduction of dimensionality and recognition

## Exercise 1: Direct PCA method

The primary purpose of the PCA method is reduction of dimensionality. This means that
we want to find such a linear transformation of the input data that will describe the data
in a low dimensional orthogonal space (which can also be called a subspace) while losing
the minimal amount of information needed to reconstruct the data. A subspace is defined
by its basis vectors.

## Exercise 2: The dual PCA method

When analyzing image data, the dimensionality can become very large (much larger than
the number of elements in the input), the direct method of computing eigenvectors we
used previously is no longer suitable. E.g. if we have 10000-dimensional data, this would
produce a covariance matrix of size 1000010000. Here we are close to hitting the limits of
computer memory. As the number of data samples N is lower than the dimensionality we
can use the dual PCA method.

## Exercise 3: Image decomposition examples

Here we will use the dual PCA method on a computer vision problem. In the supplementary
material there are three series of images. Each series contains 64 images of a face
under different lighting conditions. Your task will be to analyze each series using the PCA
method.